__Webscrape Human Genome Dataset__

- In order to map the population codes to each individual sample I went on to the Genome_Website and scraped the population samples along with their corresponding population code. 

In [3]:
!pip install selenium
!pip install bs4

  Stored in directory: /Users/AlexGaujean/Library/Caches/pip/wheels/a0/b0/b2/4f80b9456b87abedbc0bf2d52235414c3467d8889be38dd472
Successfully built bs4


In [1]:
#install necessary modules
import time
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import requests

In [2]:
#save variables for each row in our main population genome table  
page = requests.get('http://www.internationalgenome.org/cell-lines-and-dna-coriell')
soup = BeautifulSoup(page.content, 'lxml') 

popcode = soup.find('tr')
popcode = soup.findAll('tr')
#collect the more specific population descriptions 
population_descriptions = [x.td.text for x in popcode[1:]]
#collect URLs for individual sample names
hrefs = [x.td.next_sibling.next_sibling.next_sibling.next_sibling.a['href'] for x in popcode[1:]]
#collect the population codes for each of our subsamples
population_code = [x.td.next_sibling.next_sibling.text for x in popcode[1:]]

In [3]:
#create a population_dict where we will store our dictionary values and zip that with the corresponding 'href' tags that we found online
population_dict = {}
zipped_list = zip(population_code, hrefs)
list(zipped_list)[0][0]

'FIN'

In [4]:
#create a for loop that will go through our reference genome website and scrape all of the individual sample accounts
population_dict = {}
zipped_list = zip(population_code, hrefs)
#setup chrome driver to scrape the tables for population codes
browser = webdriver.Chrome('/Users/AlexGaujean/Desktop/chromedriver')
time.sleep(5)
# loop through our list of links
for i in zipped_list:
    browser.get(i[1])
    ## code for webscraping with selenium ##
    table = browser.find_element_by_xpath('//*[@id="grdRef"]')
        #get all of the rows in the table
    try: 
        rows = table.find_elements_by_tag_name('tr')
        #setup list called sample which we are going to use to append our sample_codes
        sample = []
        for row in rows[1:]:
            cols = row.find_elements_by_tag_name('td')
            sample.append(cols[0].text)
    except IndexError: 
        rows = table.find_elements_by_tag_name('tr')
        sample = []
        for row in rows[3:]:
            cols = row.find_elements_by_tag_name('td')
            sample.append(cols[0].text) 
    population_dict[i[0]] = sample

In [5]:
#filter out all the webscraped tables that skipped values 
def popcode_filter(dictionary_key): 
    dictionary_key = list(filter(lambda x: not (x == " "), dictionary_key))
popcode_filter(population_dict['MXL'])
popcode_filter(population_dict['ASW'])
popcode_filter(population_dict['YRI']) 

In [6]:
#Population description will equal the list of samples from our dictionary this way we can save them and map them onto our main file
Finnish_in_Finland = population_dict['FIN']